# Programação Matemática (GCC118)
## Universidade Federal de Lavras (UFLA)
### Instituto de Ciências Exatas e Tecnológicas

#### Grupo
- Ranulfo Mascari Neto
- Heitor Rodrigues Sabino

## Problema 1
A Empresa de Viação Aérea Brasileira está estudando a compra de três tipos de aviões: Boeing 717 para as pontes aéreas a curta distância, Boeing 737-500 para voos domésticos e internacionais de média distância e MD-11 para voos internacionais de longa distância. Em um estudo preliminar, considerou-se que a capacidade máxima dos aviões a serem comprados será sempre preenchida para efeito de planejamento. Os dados de planejamento constam na Tabela 1.


**Tabela 1: Informações sobre os aviões.**

|        Avião        | Custo (milhões de \$) | Receita Teórica (milhões de \$) | Pilotos aptos |
|:-------------------:|:---------------------:|:-------------------------------:|:-------------:|
|     BOEING 717     |          5,1          |              330                |       30      |
|   BOEING 737-500   |          3,6          |              300                |       20      |
|        MD-11       |          6,8          |              420                |       10      |


A verba disponível para as compras é de 220 milhões de dólares. Os pilotos de MD-11 podem pilotar todos os aviões da empresa, mas os demais pilotos só podem ser escalados às aeronaves a que foram habilitados. Cada aeronave necessita de dois pilotos para operar. As oficinas de manutenção podem suportar até 40 Boeings 717. Um Boeing 737-500 equivale, em esforço de manutenção, a $\frac{3}{4}$, e um MD-11 a $\frac{5}{3}$, quando referidos ao Boeing 717.

### Objetivo

Determinar as aquisições de aviões nesta empresa, maximizando as receitas.


### Instalação da biblioteca PuLP

Para mais informações, acesse: [PuLP 2.9.0](https://pypi.org/project/PuLP/).

In [1]:
!pip install pulp
import pulp

## Modelagem Matemática

A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão.

### Parâmetros

- **Custo**:
  - $ c_i \in \mathbb{R}_+ $: custo do avião do modelo $ i $, onde $ i \in A $.  
- **Receita Teórica**:
  - $ r_i \in \mathbb{R}_+ $: receita teórica do avião do modelo $ i $, onde $ i \in A $.  
- **Pilotos Aptos**:
  - $ p_i \in \mathbb{Z}_+ $: número de pilotos aptos para operar o avião do modelo $ i $, onde $ i \in A $.  
- **Esforço de Manutenção**:
  - $ e_i \in \mathbb{R}_+ $: esforço de manutenção requerido para o avião do modelo $ i $, definido em relação ao modelo $ j = 0 $  (Boeing 717), onde $ i, j \in A $.  
- **Verba Disponível**:
  - $ B \in \mathbb{R}_+ $: verba disponível para aquisição dos aviões, sendo $ B $ = 220 milhões de dólares.
- **Limite de Manutenção**:
  - $ M_{max} \in \mathbb{Z}_+ $: limite máximo de aviões que podem ser mantidos, equivalente a 40 Boeings 717.


- **Aviões**:
  - $ A $: conjunto de modelos de aviões, onde $ i \in A $ representa:
    1. **Boeing 717** ($ i = 0 $):
      - Custo ($ c_0 $): 5,1 milhões de dólares
      - Receita Teórica ($ r_0 $): 330 milhões de dólares
      - Pilotos Aptos ($ p_0 $): 30
      - Esforço de Manutenção ($ e_{0} $): 1

    2. **Boeing 737-500** ($ i = 1 $):
      - Custo ($ c_1 $): 3,6 milhões de dólares
      - Receita Teórica ($ r_1 $): 300 milhões de dólares
      - Pilotos Aptos ($ p_1 $): 20
      - Esforço de Manutenção ($ e_{1} $): $\frac{3}{4}$

    3. **MD-11** ($ i = 2 $):
      - Custo ($ c_2 $): 6,8 milhões de dólares
      - Receita Teórica ($ r_2 $): 420 milhões de dólares
      - Pilotos Aptos ($ p_2 $): 10
      - Esforço de Manutenção ($ e_{2} $): $\frac{5}{3}$

In [2]:
A = ["Boeing 717", "Boeing 737-500", "MD-11"]

c = {
    "Boeing 717": 5.1,
    "Boeing 737-500": 3.6,
    "MD-11": 6.8
}

r = {
    "Boeing 717": 330,
    "Boeing 737-500": 300,
    "MD-11": 420
}

p = {
    "Boeing 717": 30,
    "Boeing 737-500": 20,
    "MD-11": 10
}

e = {
    "Boeing 717": 1,
    "Boeing 737-500": 3/4,
    "MD-11": 5/3
}

B = 220
M_max = 40

### Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem('aquisicao_avioes', pulp.LpMaximize)

### Variáveis de Decisão

- $x_i \geq 0$: representa o número de aquisições do avião de modelo $i$ pela empresa, onde $i \in A$.


In [4]:
x = pulp.LpVariable.dicts('x', A, lowBound=0, cat='Integer')

### Função Objetivo

- Maximização da Receita: $\max \sum_{i \in A} r_ix_i$

  onde:
  - $x_i$ é o número de aquisições do avião de modelo $i$;
  - $r_i$ é a receita teórica gerada pelo avião de modelo $i$.


In [5]:
modelo += pulp.lpSum([r[i] * x[i] for i in A])

### Restrições

As restrições do problema são as seguintes:

- **Verba Disponível**: A soma dos custos dos aviões adquiridos não pode exceder a verba disponível.

  $$\sum_{i \in A} c_i x_i \leq B$$

In [6]:
modelo += pulp.lpSum([c[i] * x[i] for i in A]) <= B

- **Limite de Manutenção**: O esforço total de manutenção requerido não pode exceder o limite de manutenção disponível. O esforço total de manutenção é calculado em relação ao Boeing 717.

  $$\sum_{i \in A} e_i x_i \leq M_{\text{max}}$$

In [7]:
modelo += pulp.lpSum([e[i] * x[i] for i in A]) <= M_max

- **Capacidade de Pilotos**: A quantidade de pilotos deve atender às exigências de cada modelo de avião, considerando que cada avião requer 2 pilotos:

  - Para a aeronave Boeing 717, os pilotos de Boeing 717 e MD-11 podem ser escalados:

  $$2x_0 \leq p_0 + p_2$$

  - Para a aeronave Boeing 737-500, os pilotos de Boeing 737-500 e MD-11 podem ser escalados:

  $$2x_1 \leq p_1 + p_2$$

  - Somente os pilotos de MD-11 podem ser escalados à aeronave MD-11:

  $$2x_2 \leq p_2$$

In [8]:
for i in A:
  if i == "MD-11":
    modelo += 2 * x[i] <= p[i]
  else:
    modelo += 2 * x[i] <= p[i] + p["MD-11"]

- **Não Negatividade**: O número de aquisições de cada modelo de avião deve ser não negativo.

  $$x_i \geq 0, \quad \forall i \in A$$

In [9]:
modelo

aquisicao_avioes:
MAXIMIZE
330*x_Boeing_717 + 300*x_Boeing_737_500 + 420*x_MD_11 + 0
SUBJECT TO
_C1: 5.1 x_Boeing_717 + 3.6 x_Boeing_737_500 + 6.8 x_MD_11 <= 220

_C2: x_Boeing_717 + 0.75 x_Boeing_737_500 + 1.66666666667 x_MD_11 <= 40

_C3: 2 x_Boeing_717 <= 40

_C4: 2 x_Boeing_737_500 <= 30

_C5: 2 x_MD_11 <= 10

VARIABLES
0 <= x_Boeing_717 Integer
0 <= x_Boeing_737_500 Integer
0 <= x_MD_11 Integer

### Resolvendo o problema

In [10]:
status = modelo.solve()

### Imprimindo as soluções do problema

In [11]:
print('status: ', pulp.LpStatus[status])
print('funcao objetivo: ', modelo.objective.value())

for i in A:
    print(f"Quantidade de {i} comprados: {x[i].varValue}")
print(f"Receita prevista: {modelo.objective.value()} milhões de dólares.")

status:  Optimal
funcao objetivo:  13200.0
Quantidade de Boeing 717 comprados: 20.0
Quantidade de Boeing 737-500 comprados: 15.0
Quantidade de MD-11 comprados: 5.0
Receita prevista: 13200.0 milhões de dólares.
